In [64]:
import numpy as np
 
def calc_MI(X,Y,bins):
 
    c_XY = np.histogram2d(X,Y,bins)[0]
    c_X = np.histogram(X,bins)[0]
    c_Y = np.histogram(Y,bins)[0]
 
    H_X = shan_entropy(c_X)
    H_Y = shan_entropy(c_Y)
    H_XY = shan_entropy(c_XY)
 
    MI = H_X + H_Y - H_XY
    return MI
 
def shan_entropy(c):
    c_normalized = c / float(np.sum(c))
    c_normalized = c_normalized[np.nonzero(c_normalized)]
    H = -sum(c_normalized* np.log2(c_normalized))  
    return H
 
A = np.array([[ 2.0,  140.0,  128.23, -150.5, -5.4  ],
              [ 2.4,  153.11, 130.34, -130.1, -9.5  ],
              [ 1.2,  156.9,  120.11, -110.45,-1.12 ]])
 
bins = 5 # ?
n = A.shape[1]
matMI = np.zeros((n, n))
 
for ix in np.arange(n):
    for jx in np.arange(ix+1,n):
        matMI[ix,jx] = calc_MI(A[:,ix], A[:,jx], bins)
matMI

array([[0.       , 1.5849625, 1.5849625, 1.5849625, 1.5849625],
       [0.       , 0.       , 1.5849625, 1.5849625, 1.5849625],
       [0.       , 0.       , 0.       , 1.5849625, 1.5849625],
       [0.       , 0.       , 0.       , 0.       , 1.5849625],
       [0.       , 0.       , 0.       , 0.       , 0.       ]])

In [65]:
from MI_score import *
X, y = shap.datasets.california(n_points=1000)
X100 = shap.utils.sample(X, 100)
X

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
14740,4.1518,22.0,5.663073,1.075472,1551.0,4.180593,32.58,-117.05
10101,5.7796,32.0,6.107226,0.927739,1296.0,3.020979,33.92,-117.97
20566,4.3487,29.0,5.930712,1.026217,1554.0,2.910112,38.65,-121.84
2670,2.4511,37.0,4.992958,1.316901,390.0,2.746479,33.20,-115.60
15709,5.0049,25.0,4.319261,1.039578,649.0,1.712401,37.79,-122.43
...,...,...,...,...,...,...,...,...
13339,5.7530,14.0,6.071023,0.980114,1151.0,3.269886,34.03,-117.66
2791,1.8325,25.0,4.279221,1.070130,1477.0,1.918182,37.37,-118.39
1550,6.8806,16.0,8.273632,1.042289,1272.0,3.164179,37.74,-121.93
17652,3.8936,13.0,4.128079,1.160099,710.0,1.748768,37.26,-121.88


In [66]:
from sklearn.feature_selection import mutual_info_regression
# determine the mutual information
mutual_info = mutual_info_regression(X, y)
mutual_info = pd.Series(mutual_info)
mutual_info.index = X.columns
mutual_info.sort_values(ascending=False)

MedInc        0.339933
Longitude     0.287627
Latitude      0.241211
AveRooms      0.127409
AveOccup      0.099220
HouseAge      0.028236
AveBedrms     0.020072
Population    0.000000
dtype: float64

In [72]:
from scipy.stats.contingency import crosstab
from sklearn.metrics import mutual_info_score
a = ['A', 'B', 'A', 'A', 'B', 'B', 'A', 'A', 'B', 'B']
x = ['X', 'X', 'X', 'Y', 'Z', 'Z', 'Y', 'Y', 'Z', 'Z']
c = crosstab(a, x)
print(c[1])
mutual_info_score(labels_true=None, labels_pred=None, contingency = c[1])

[[2 3 0]
 [1 0 4]]


0.5021929300715018

In [71]:
from scipy.stats import chi2_contingency
X_arr = np.array(X)
y_arr = np.array(y)
bins = int(np.sqrt(len(X_arr)/5))
c_xy = np.histogram2d(X_arr[:, 0], y_arr, bins=13)[0]
print(c_xy)
g, p, dof, expected = chi2_contingency(c_xy, lambda_="log-likelihood")
mi = 0.5 * g / c_xy.sum()
mi

[[27. 36. 15.  7.  4.  3.  1.  0.  3.  0.  0.  0.  0.]
 [32. 72. 55. 48. 25. 10.  8.  4.  9.  0.  3.  1.  3.]
 [ 4. 33. 59. 54. 52. 24. 19. 10. 11.  9.  3.  1.  8.]
 [ 0.  6. 18. 36. 27. 31. 18.  5. 12. 10.  4.  3.  4.]
 [ 0.  0.  2.  7. 12. 17. 13.  8.  9.  5.  2.  5.  3.]
 [ 0.  0.  0.  0.  1.  2.  6.  5.  7.  2.  3.  2. 15.]
 [ 0.  0.  0.  0.  0.  1.  4.  1.  1.  3.  6.  3.  6.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  3.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  6.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  3.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  4.]]


0.3531702431453488

In [ ]:
bins = int(np.sqrt(len(X_arr)/5))
hist2D = np.histogram2d(X_arr[:, 0], y_arr, bins)[0]
probabilities = hist2D / len(X_arr[:, 0]) + 1e-100
n_variables = X.shape[-1]
n_samples = X.shape[0]

nbins = int((n_samples/5)**.5)

histograms2d = np.zeros((n_variables, n_variables, nbins, nbins))
 
for i in range(n_variables):
   for j in range(n_variables):
      histograms2d[i,j] = np.histogram2d(X_arr[:,i], X_arr[:,j], bins=nbins)[0]
probs = histograms2d / len(X) + 1e-100
joint_entropies = -(probs * np.log2(probs)).sum((2,3))
def joint_entropies(data, nbins=None):
    n_variables = data.shape[-1]
    n_samples = data.shape[0]
    if nbins == None:
        nbins = int((n_samples/5)**.5)
    histograms2d = np.zeros((n_variables, n_variables, nbins, nbins))
    for i in range(n_variables):
        for j in range(n_variables):
            histograms2d[i,j] = np.histogram2d(data[:,i], data[:,j], bins=nbins)[0]
    probs = histograms2d / len(data) + 1e-100
    joint_entropies = -(probs * np.log2(probs)).sum((2,3))
    return joint_entropies
j_entropies = joint_entropies(X_arr)
print(j_entropies)
entropies = j_entropies.diagonal()
entropies_tile = np.tile(entropies, (n_variables, 1)).T
sum_entropies = entropies_tile + entropies_tile.T
mi_matrix = sum_entropies - j_entropies
norm_mi_matrix = mi_matrix * 2 / sum_entropies
mi_matrix

In [54]:
def joint_entropies(data, nbins=None):
    n_variables = data.shape[-1]
    n_samples = data.shape[0]
    if nbins == None:
        nbins = int((n_samples/5)**.5)
    histograms2d = np.zeros((n_variables, n_variables, nbins, nbins))
    for i in range(n_variables):
        for j in range(n_variables):
            histograms2d[i,j] = np.histogram2d(data[:,i], data[:,j], bins=nbins)[0]
    probs = histograms2d / len(data) + 1e-100
    joint_entropies = -(probs * np.log2(probs)).sum((2,3))
    return joint_entropies

def mutual_info_matrix(df, nbins=None, normalized=False):
    data = df.to_numpy()
    n_variables = data.shape[-1]
    j_entropies = joint_entropies(data, nbins)
    entropies = j_entropies.diagonal()
    entropies_tile = np.tile(entropies, (n_variables, 1))
    sum_entropies = entropies_tile + entropies_tile.T
    mi_matrix = sum_entropies - j_entropies
    if normalized:
        mi_matrix = mi_matrix * 2 / sum_entropies    
    return pd.DataFrame(mi_matrix, index=df.columns, columns=df.columns)
f = X[["MedInc"]]
f["Target"] = y
mutual_info_matrix(X)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
MedInc,2.685326,0.132804,0.286153,0.040064,0.077564,0.059763,0.147960,0.147550
HouseAge,0.132804,3.700529,0.078709,0.057961,0.171548,0.070662,0.190012,0.264894
AveRooms,0.286153,0.078709,1.163814,0.152644,0.044737,0.082692,0.072602,0.110996
AveBedrms,0.040064,0.057961,0.152644,0.403052,0.046779,0.025830,0.063782,0.104022
Population,0.077564,0.171548,0.044737,0.046779,2.255044,0.079090,0.083606,0.104725
AveOccup,0.059763,0.070662,0.082692,0.025830,0.079090,1.577784,0.076642,0.098690
Latitude,0.147960,0.190012,0.072602,0.063782,0.083606,0.076642,3.004677,1.516527
Longitude,0.147550,0.264894,0.110996,0.104022,0.104725,0.098690,1.516527,3.162778
